In [1]:
import torch
from torch.nn import functional as F
import numpy as np
# from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm
from rsna19.configs.second_level import Config
from sklearn.metrics import log_loss
import pandas as pd
from scipy.signal import windows

In [2]:
# train_folds = [0, 1, 2, 3] # for testing 
train_folds = [0, 1, 2, 3, 4]  # for stage 2 submission
val_folds = [4]

cache_dir = '/home/dmytro/ml/kaggle-rsna-2019/output/cache_stage2'

train_x = torch.cat([torch.tensor(np.load(f'{cache_dir}/fold{f}/x.npy'), dtype=torch.float32) for f in train_folds], dim=0)
train_y = torch.cat([torch.tensor(np.load(f'{cache_dir}/fold{f}/y.npy'), dtype=torch.float32) for f in train_folds], dim=0)
val_x = torch.cat([torch.tensor(np.load(f'{cache_dir}/fold{f}/x.npy'), dtype=torch.float32) for f in val_folds], dim=0)
val_y = torch.cat([torch.tensor(np.load(f'{cache_dir}/fold{f}/y.npy'), dtype=torch.float32) for f in val_folds], dim=0)

n_models = 11
class_weights = torch.tensor([1, 1, 1, 1, 1, 2], dtype=torch.float32) * 6 / 7
loss_fn = F.binary_cross_entropy

In [3]:
# undo sigmoid

# train_x[train_x > 0] = torch.log(train_x[train_x > 0] / (1-train_x[train_x > 0]))
# val_x[val_x > 0] = torch.log(val_x[val_x > 0] / (1-val_x[val_x > 0]))
# loss_fn = F.binary_cross_entropy_with_logits

In [4]:
print('train')
preds = []
for model_id in range(n_models):
    preds.append(train_x[:, model_id*30+12:model_id*30+18])
    loss = loss_fn(preds[-1], train_y, weight=class_weights)
    print(f'model {model_id}: {loss}')

mean_preds = torch.mean(torch.stack(preds), dim=0)
loss = loss_fn(mean_preds, train_y, weight=class_weights)
print(f'averaged ensemble: {loss}')

print('\nval')
preds = []
for model_id in range(n_models):
    preds.append(val_x[:, model_id*30+12:model_id*30+18])
    loss = loss_fn(preds[-1], val_y, weight=class_weights)
    print(f'model {model_id}: {loss}')

mean_preds = torch.mean(torch.stack(preds), dim=0)
loss = loss_fn(mean_preds, val_y, weight=class_weights)
print(f'averaged ensemble: {loss}')

train
model 0: 0.06121179461479187
model 1: 0.06256157159805298
model 2: 0.06239280849695206
model 3: 0.06186671555042267
model 4: 0.06123921647667885
model 5: 0.06273935735225677
model 6: 0.06325805187225342
model 7: 0.060901932418346405
model 8: 0.060555413365364075
model 9: 0.06571821123361588
model 10: 0.06511762738227844
averaged ensemble: 0.05699021369218826

val
model 0: 0.061941273510456085
model 1: 0.06328877061605453
model 2: 0.06281528621912003
model 3: 0.06388996541500092
model 4: 0.06146755814552307
model 5: 0.06464578211307526
model 6: 0.06419891119003296
model 7: 0.0613483302295208
model 8: 0.0610339418053627
model 9: 0.06970273703336716
model 10: 0.06656711548566818
averaged ensemble: 0.05792373791337013


In [5]:
# train on middle slice only

# train_x_1slice = []
# for model_id in range(n_models):
#     train_x_1slice.append(train_x[:, model_id*30+12:model_id*30+18])
# train_x_1slice = torch.cat(train_x_1slice, dim=1)

# val_x_1slice = []
# for model_id in range(n_models):
#     val_x_1slice.append(val_x[:, model_id*30+12:model_id*30+18])
# val_x_1slice = torch.cat(val_x_1slice, dim=1)

# train_x = train_x_1slice
# val_x = val_x_1slice

In [6]:
hidden = 128
features_out = 6

# model = torch.nn.Sequential(
#         torch.nn.Linear(train_x.shape[1], features_out)
# )

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.w1 = torch.nn.Linear(train_x.shape[1], features_out, bias=False)

    def forward(self, x):
        x = F.linear(x, torch.abs(self.w1.weight) / torch.sum(torch.abs(self.w1.weight), 1, keepdim=True), self.w1.bias)
#         x = F.linear(x, self.w1.weight / torch.sum(self.w1.weight, 1, keepdim=True), self.w1.bias)
        return torch.clamp(x, 0, 1)


model = Model()
print(model.w1.weight.shape)
# model = torch.nn.Sequential(
#         torch.nn.Linear(train_x.shape[1], hidden),
#         torch.nn.ReLU(),
# #         torch.nn.Dropout(0.2),
#         torch.nn.Linear(hidden, features_out),
# )

train_x = train_x.cuda()
train_y = train_y.cuda()
val_x = val_x.cuda()
val_y = val_y.cuda()
model = model.cuda()

optimizer = torch.optim.Adam(model.parameters(), 0.0001)
val_log_loss = 0
class_weights = class_weights.cuda()

for i in tqdm(range(10000)):
    optimizer.zero_grad()

    y_hat = model(train_x)
    loss = F.binary_cross_entropy(y_hat, train_y, weight=class_weights)
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        model.eval()
        val_y_hat = model(val_x)
        val_loss = F.binary_cross_entropy(val_y_hat, val_y, weight=class_weights)
        model.train()
        
        print(f'{i:04d}: train: {loss.item():.04f}, val: {val_loss.item():.04f}')

torch.Size([6, 330])


  0%|          | 22/10000 [00:00<01:36, 103.51it/s]

0000: train: 0.1286, val: 0.1292


  1%|          | 121/10000 [00:01<01:36, 102.66it/s]

0100: train: 0.1133, val: 0.1138


...

 98%|█████████▊| 9812/10000 [01:36<00:01, 100.84it/s]

9800: train: 0.0570, val: 0.0575


 99%|█████████▉| 9911/10000 [01:37<00:00, 100.84it/s]

9900: train: 0.0570, val: 0.0575


100%|██████████| 10000/10000 [01:38<00:00, 101.82it/s]


In [14]:
torch.abs(model.w1.weight) / torch.sum(torch.abs(model.w1.weight), 1, keepdim=True)

tensor([[1.9564e-06, 2.1241e-06, 3.5336e-08,  ..., 1.7863e-06, 5.2180e-06,
         3.6598e-06],
        [1.6304e-06, 3.6274e-06, 6.9784e-06,  ..., 1.2331e-05, 9.9599e-06,
         4.8471e-06],
        [6.4633e-06, 7.1205e-06, 9.2582e-07,  ..., 7.6014e-06, 1.0359e-05,
         4.1163e-06],
        [1.2718e-05, 4.5825e-06, 9.0938e-07,  ..., 1.1724e-05, 5.2204e-06,
         7.0850e-06],
        [5.4611e-06, 1.0649e-05, 5.8644e-06,  ..., 1.0276e-05, 4.7437e-06,
         3.5321e-06],
        [2.3052e-06, 2.2436e-06, 1.1978e-06,  ..., 3.3766e-06, 1.2293e-05,
         1.9789e-06]], device='cuda:0', grad_fn=<DivBackward0>)

Apply the trained model on the L1 test predictions:

TODO: implement apply to test and call generate_submission with clip_eps 1e-5

Load test prediction of the same level 1 models and combine using trained l2 model:

In [27]:
test_x = np.mean([np.load(f'{cache_dir}/fold{f}/x_test.npy') for f in [0,1,2,3,4]], axis=0)

In [28]:
test_pred = model(torch.tensor(test_x, dtype=torch.float32).cuda()).cpu().detach().numpy()

In [29]:
# load one of the l1 model predictions as the base for l2 csv file:
import glob
config = Config()
pred1_path = sorted(glob.glob(f'/home/dmytro/ml/kaggle-rsna-2019/output/prediction/{config.models[-1]}/fold1/predictions_stage2/test_*.csv'))[0]
pred1 = pd.read_csv(pred1_path)
pred1_path

'/home/dmytro/ml/kaggle-rsna-2019/output/prediction/se_preresnext26b_400/fold1/predictions_stage2/test_h_flip.csv'

In [30]:
pred1 = pred1.sort_values(by=['study_id', 'slice_num'])

In [31]:
for i,c in enumerate(config.pred_columns):
    pred1[c] = test_pred[:, i]

In [32]:
pred1.to_csv('/home/dmytro/ml/kaggle-rsna-2019/output/pred_l2.csv', index=False)